In [1]:
import os

import numpy as np
from scipy.sparse import load_npz
from scipy.stats import pearsonr
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import pandas as pd

import torch
import torch_geometric
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch_geometric.nn.conv import GATConv

import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
cell_line = 'E116'
regression_flag = 0
chip_res = 10000
hic_res = 10000
num_hm = 6
num_feat = int((hic_res/chip_res)*num_hm)
num_classes = 2 if regression_flag == 0 else 1

In [4]:
#define data paths
src_dir = os.getcwd()
#src_dir = os.path.dirname(base_path)
save_dir = os.path.join(src_dir, 'data', cell_line, 'saved_runs')
hic_sparse_mat_file = os.path.join(src_dir, 'data', cell_line, 'hic_sparse.npz')
np_nodes_lab_genes_file = os.path.join(src_dir, 'data',  cell_line, \
    'np_nodes_lab_genes_reg' + str(regression_flag) + '.npy')
np_hmods_norm_all_file = os.path.join(src_dir, 'data', cell_line, \
    'np_hmods_norm_chip_' + str(chip_res) + 'bp.npy')
df_genes_file = os.path.join(src_dir, 'data', cell_line, 'df_genes_reg' + str(regression_flag) + '.pkl')

In [5]:
df_genes = pd.read_pickle(df_genes_file)
df_genes

,gene_catalog_name,abbrev,hic_node_id,expression_lvl,connected
0,ENSG00000237613,FAM138A,3,0,1.0
1,ENSG00000186092,OR4F5,6,0,1.0
2,ENSG00000235249,OR4F29,36,0,1.0
3,ENSG00000185097,OR4F16,62,0,1.0
4,ENSG00000197049,NaN,72,1,1.0
...,...,...,...,...,...
384,ENSG00000100288,CHKB,279585,0,1.0
385,ENSG00000100299,ARSA,279588,1,1.0
386,ENSG00000251322,SHANK3,279593,0,1.0
387,ENSG00000100312,ACR,279599,0,1.0


In [6]:
mat = load_npz(hic_sparse_mat_file)
allNodes_hms = np.load(np_hmods_norm_all_file) #contains 6 histone marks for all 279606 regions + id (Shape = [279606, 7])
hms = allNodes_hms[:, 1:] #only includes features, not node ids (Shape = [279606, 6])
X = torch.tensor(hms).float().reshape(-1, num_feat) #convert hms to tensor (Shape = [279606, 6])
allNodes = allNodes_hms[:, 0].astype(int) #contains ids of all regions (Shape = [279606, 1])


geneNodes_labs = np.load(np_nodes_lab_genes_file) #contains the expression level of each gene (Shape = [16699, 2])

#Dataframe only contains regions that encode a gene
print(f'{(geneNodes_labs.shape[0] * 100 / X.shape[0]):.2f}% of all regions encode genes\n')
pd.DataFrame(geneNodes_labs, columns=['gene_id', 'gene_expressed?']).set_index('gene_id').head()

5.97% of all regions encode genes



,gene_expressed?
gene_id,
3.0,0.0
6.0,0.0
36.0,0.0
62.0,0.0
72.0,1.0


In [7]:
geneNodes = geneNodes_labs[:, -2].astype(int) #contains ids of regions that encode a gene (Shape = [16699, 1])

allLabs = -1*np.ones(np.shape(allNodes))
targetNode_mask = torch.tensor(geneNodes).long()
geneLabs = geneNodes_labs[:, -1].astype(int)
allLabs[geneNodes] = geneLabs #contains expression level for each region (-1 if region doesn't encode gene, 1 if gene is expressed, 0 if not)
Y = torch.tensor(allLabs).long()

In [8]:
extract = torch_geometric.utils.from_scipy_sparse_matrix(mat)

#extract[0] (Shape = [2, 3906914]) contains nodes that have edges between them --> (extract[0][0][0], extract[0][1][0]) is an edge
#extract[1] (Shape = [1, 3906914]) contain the HI-C edge weights
extract

(tensor([[     0,      1,      1,  ..., 279605, 279605, 279605],
         [  8765,     24,     67,  ..., 279595, 279598, 279599]]),
 tensor([168.5447,  66.0712,  15.8497,  ...,  38.9206,  59.0113, 120.2268],
        dtype=torch.float64))

In [9]:
#define graph with nodes, node features, edges, and edge features

data = torch_geometric.data.Data(edge_index = extract[0], edge_attr = extract[1], x = X, y = Y)
G = data
G

Data(x=[279606, 6], edge_index=[2, 3906914], edge_attr=[3906914], y=[279606])

In [10]:
pred_idx_shuff = torch.randperm(targetNode_mask.shape[0])
fin_train = np.floor(0.7*pred_idx_shuff.shape[0]).astype(int)
fin_valid = np.floor(0.85*pred_idx_shuff.shape[0]).astype(int)
train_idx = pred_idx_shuff[:fin_train]
valid_idx = pred_idx_shuff[fin_train:fin_valid]
test_idx = pred_idx_shuff[fin_valid:]

In [11]:
def to_cpu_npy(x):
    return x.cpu().detach().numpy()
    
def train_model_classification(model, loss, graph, max_epoch, learning_rate, targetNode_mask, train_idx, valid_idx, optimizer):
    model = model.to(device)
    graph = graph.to(device)

    optimizer = optimizer
    
    train_labels = to_cpu_npy(graph.y[targetNode_mask[train_idx]])
    valid_labels = to_cpu_npy(graph.y[targetNode_mask[valid_idx]])
    
    model.train()
    train_status = True
    
    print('\n')

    train_losses = []
    valid_losses = []
    for e in list(range(max_epoch)):
        model.train()
        optimizer.zero_grad()

        all_scores = model(graph)[targetNode_mask]
        train_scores = all_scores[train_idx]
        
        train_loss = loss(train_scores, torch.LongTensor(train_labels).to(device))
        train_losses.append(train_loss.item())

        train_loss.backward()
        optimizer.step()

        model.eval()
        valid_scores = all_scores[valid_idx]
        valid_loss = loss(valid_scores, torch.LongTensor(valid_labels).to(device))
        valid_losses.append(valid_loss.item())

        if e%100 == 0:
            print(f'Epoch {e}: Train Loss = {train_loss}, Valid Loss = {valid_loss}')

    return train_losses, valid_losses

def eval_model_classification(model, graph, targetNode_mask, train_idx, valid_idx, test_idx):
    model = model.to(device)
    graph = graph.to(device)
    test_labels = to_cpu_npy(graph.y[targetNode_mask[test_idx]])
    
    model.eval()

    forward_scores = model(G)[targetNode_mask]

    test_scores = forward_scores[test_idx]
    test_softmax = F.softmax(test_scores, dim=1)
    test_preds = torch.argmax(test_softmax, dim=1)
    
    test_softmax = to_cpu_npy(test_softmax)
    test_preds = to_cpu_npy(test_preds)
    test_AUROC = roc_auc_score(test_labels, test_softmax[:,1], average="micro")
    test_acc = np.mean(test_preds == test_labels)

    train_labels = to_cpu_npy(graph.y[targetNode_mask[train_idx]])
    train_scores = forward_scores[train_idx]
    train_softmax = F.softmax(train_scores, dim=1)
    train_preds = torch.argmax(train_softmax, dim=1)
    
    train_softmax = to_cpu_npy(train_softmax)
    train_preds = to_cpu_npy(train_preds)
    train_AUROC = roc_auc_score(train_labels, train_softmax[:,1], average="micro")
    train_acc = np.mean(train_preds == train_labels)


    return {'train_AUROC': train_AUROC, 'train_acc': train_acc, 'test_AUROC': test_AUROC, 'test_acc': test_acc}


In [12]:
class GAT(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_heads=3):
        super(GAT, self).__init__()
        self.gat1 = GATConv(in_channels, hidden_channels[0], heads=num_heads, concat=True)
        self.gat2 = GATConv(hidden_channels[0] * num_heads, hidden_channels[1], heads=num_heads, concat=False)
        #self.gat3 = GATConv(hidden_channels[1] * num_heads, hidden_channels[2], heads=num_heads, concat=False)
        
        self.ff1 = nn.Linear(hidden_channels[-1], hidden_channels[-1] // 2)
        self.ff2 = nn.Linear(hidden_channels[-1] // 2, 2)
        self.dropout = nn.Dropout(p=0.15)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = torch.relu(self.gat1(x, edge_index, edge_attr))
        x = torch.relu(self.gat2(x, edge_index, edge_attr))
        #x = torch.relu(self.gat3(x, edge_index, edge_attr))
        x = self.dropout(x)
        x = torch.relu(self.ff1(x))
        x = self.ff2(x)
        
        return x

In [13]:
learning_rate = 1e-3
max_epoch = 1400
loss = nn.CrossEntropyLoss()

gat = GAT(in_channels=6, hidden_channels=[10, 45])
optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, gat.parameters()), lr = learning_rate)

train_losses, valid_losses = train_model_classification(gat, loss, G, max_epoch, learning_rate, targetNode_mask, train_idx, valid_idx, optimizer)



Epoch 0: Train Loss = 0.6947699189186096, Valid Loss = 0.694015383720398
Epoch 100: Train Loss = 0.5376247763633728, Valid Loss = 0.5418145060539246
Epoch 200: Train Loss = 0.5134361386299133, Valid Loss = 0.5159600377082825
Epoch 300: Train Loss = 0.4899270236492157, Valid Loss = 0.4910199046134949
Epoch 400: Train Loss = 0.46335020661354065, Valid Loss = 0.46137723326683044
Epoch 500: Train Loss = 0.44814741611480713, Valid Loss = 0.44604140520095825
Epoch 600: Train Loss = 0.4391421675682068, Valid Loss = 0.43929585814476013
Epoch 700: Train Loss = 0.432937890291214, Valid Loss = 0.4341385066509247
Epoch 800: Train Loss = 0.42843005061149597, Valid Loss = 0.4361973702907562
Epoch 900: Train Loss = 0.423936665058136, Valid Loss = 0.43461519479751587
Epoch 1000: Train Loss = 0.421219140291214, Valid Loss = 0.43460676074028015
Epoch 1100: Train Loss = 0.4181046485900879, Valid Loss = 0.42998406291007996
Epoch 1200: Train Loss = 0.4160389006137848, Valid Loss = 0.429818332195282
Epoch

In [14]:
out = eval_model_classification(gat, G, targetNode_mask, train_idx, valid_idx, test_idx)

In [15]:
out

{'train_AUROC': 0.8946323743687599,
 'train_acc': 0.821541620326803,
 'test_AUROC': 0.8806864680729176,
 'test_acc': 0.8099800399201597}